In [1]:
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import pandas as pd
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from tpot import TPOTClassifier
from tpot import TPOTRegressor
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore")
import re
from textblob import TextBlob

/Users/rlieu/Library/Python/3.9/lib/python/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


LOCALLY

GitHub

In [2]:
# # condense into one df 
df1 = pd.read_csv('../Datasets/aspergers.csv', parse_dates=True)
df2 = pd.read_csv('../Datasets/depression.csv', parse_dates=True)
df3 = pd.read_csv('../Datasets/ocd.csv', parse_dates=True)
df4 = pd.read_csv('../Datasets/ptsd.csv', parse_dates=True)
df5 = pd.read_csv('../Datasets/adhd.csv', parse_dates=True)

df = pd.concat([df1, df2, df3, df4,df5], ignore_index=True)
#LOWER ALL subreddit 
df['subreddit'] = df['subreddit'].str.lower()
df.head()

,author,body,created_utc,id,num_comments,score,subreddit,title,upvote_ratio,url
0,Adadum,I wish I could do a poll for this but what wou...,2021-12-23T17:56:36.000Z,rn19yc,0,1,aspergers,Separatism for high-functioning Autists?,1.0,https://www.reddit.com/r/aspergers/comments/rn...
1,Lazy_Opposite8263,I was diagnosed with high functioning asperger...,2021-12-23T17:53:41.000Z,rn17lw,1,1,aspergers,Friend blaming toxic behaviour on autism? How ...,1.0,https://www.reddit.com/r/aspergers/comments/rn...
2,LockedOutOfElfland,It's been a while since I've seen a therapist ...,2021-12-23T17:28:07.000Z,rn0n6k,2,3,aspergers,Do therapists/mental health professionals misi...,1.0,https://www.reddit.com/r/aspergers/comments/rn...
3,PlazmaSkittles,New here. 22 years old. Diagnosed at 12. AD...,2021-12-23T16:54:23.000Z,rmzvwn,3,1,aspergers,I think I'm broken,1.0,https://www.reddit.com/r/aspergers/comments/rm...
4,xime_rollin,based on your understanding,2021-12-23T16:53:43.000Z,rmzvds,0,1,aspergers,I recently took an eq test and scored 20 out o...,1.0,https://www.reddit.com/r/aspergers/comments/rm...


In [3]:
null_subreddit_count = df['subreddit'].isnull().sum()
print(null_subreddit_count)

df['subreddit'].unique()

0


array(['aspergers', 'depression', 'ocd', 'ptsd', 'adhd'], dtype=object)

Exported sample size of post for external regex validation 

In [4]:
df_testy = df['body'].sample(n=50, random_state=1)
df_testy_df = df_testy.to_frame()

Next we will need to hot encode text variables we want to include in the model. For example Body is a post that is text based will convert this in to a sentiment score 
Also subreddit will be our target variable so we need to preform mapping on this column and then excluded the columns we will not use

In [5]:
#clean the text before performing sentiment analysis my hope is to improve optimization and accuracy improvement

def clean_text(text: str, slang_map: dict = None, stopwords = None) -> str:
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    # Remove user mentions (@username)
    text = re.sub(r'\@\w+', '', text)
    # Remove hashtags (but keep the text)
    text = re.sub(r'\#\w+', '', text)
    # Remove HTML entities (e.g., &amp;, &lt;, etc.)
    text = re.sub(r'\&\w+;', '', text)
    # Remove emojis
    emoji_pattern = re.compile("[" 
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    # whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    # Optional: Convert slang or shorthand notations to full form
    if slang_map:
        for slang, full_form in slang_map.items():
            text = text.replace(slang, full_form)
    # Optional: Remove punctuations
    text = re.sub(r'[^\w\s]', '', text)
    # Optional: Remove stopwords
    if stopwords:
        text = ' '.join([word for word in text.split() if word not in stopwords])
    # Optional: Convert text to lowercase
    text = text.lower()
    return text

In [6]:
#mapping 
subreddit_to_target = {
    'aspergers': 1,
    'depression': 2,
    'ocd': 3,
    'ptsd': 4,
    'adhd' :5,
}

df['target'] = df['subreddit'].map(subreddit_to_target)

# hot coding body to sentiment_score 
df['clean_body'] = df['body'].apply(lambda x: clean_text(str(x)))

# Apply sentiment analysis
df['sentiment_score'] = df['clean_body'].apply(lambda text: TextBlob(text).sentiment.polarity)
#setting target varibale 
test_df = df[['target','num_comments','score','upvote_ratio','sentiment_score','target']]
test_df.head()

,target,num_comments,score,upvote_ratio,sentiment_score,target
0,1,0,1,1.0,0.412500,1
1,1,1,1,1.0,-0.063263,1
2,1,2,3,1.0,0.021373,1
3,1,3,1,1.0,0.013254,1
4,1,0,1,1.0,0.000000,1


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

#mapping 
subreddit_to_target = {
    'aspergers': 1,
    'depression': 2,
    'ocd': 3,
    'ptsd': 4,
    'adhd' :5,
}

df['target'] = df['subreddit'].map(subreddit_to_target)

# hot coding body to sentiment_score 
documents_train = df['body'].apply(lambda x: clean_text(str(x)))
print(f"documents_train: {documents_train}")

# Apply topic modeling 
tfidf_vectorizer = TfidfVectorizer(
    max_features=20000,  # only top 5k by freq
    lowercase=True,  # drop capitalization
    ngram_range=(1, 1),
    min_df=2,  # note: absolute count of doc
    max_df=0.05,  # note: % of docs
    token_pattern=r"\b[a-z]{3,12}\b",  # remove short, non-word-like terms
    stop_words="english",
)  # default English stopwords

tfidf_documents = tfidf_vectorizer.fit_transform(documents_train)
feature_names = tfidf_vectorizer.get_feature_names_out()
print(f"feature_names: {feature_names}")

documents_train: 0         i wish i could do a poll for this but what wou...
1         i was diagnosed with high functioning asperger...
2         its been a while since ive seen a therapist mo...
3         new here 22 years old diagnosed at 12 adhd as ...
4                               based on your understanding
                                ...                        
151283                                              removed
151284    since the dsm was recently updated add now fal...
151285    i have a particularly worrying mental health i...
151286    i have these meds that i have to take in the m...
151287    i am supposed to be taking 18mg concerta but i...
Name: body, Length: 151288, dtype: object
feature_names: ['aaaaand' 'aaaand' 'aaron' ... 'zoophilia' 'zopiclone' 'zyprexa']


In [8]:
from gensim.sklearn_api import W2VTransformer
from sklearn.metrics.pairwise import cosine_similarity

import pickle

f = open("./text8_W2V.pickle", "rb")
text8_model = pickle.load(f)
f.close()

def topical_coherence(terms):
    embeddings = []
    for term in terms:
        try:
            embeddings.append(text8_model.transform(term)[0])
        except KeyError:
            continue

    if len(embeddings) == 0:
        return 0
    else:
        cos_sim = cosine_similarity(embeddings)
        np.fill_diagonal(cos_sim, 0)
        return cos_sim.mean()

In [9]:
from sklearn.decomposition import NMF

result = []
for n in range(2, 11):
    nmf = NMF(n_components=n, random_state=42, init="nndsvd")
    W = nmf.fit_transform(tfidf_documents)
    H = nmf.components_
    
    coherences = []
    for topic_index in range(0, n):
        top_indices = np.argsort(H[topic_index, :])[::-1]
        top_terms = []
        for term_index in top_indices[0:10]:
            top_terms.append(feature_names[term_index])
        coherences.append(topical_coherence(top_terms))

    result.append(np.median(coherences))

result

[0.17541161,
 0.22177409,
 0.21064776,
 0.22773312,
 0.26023465,
 0.27220905,
 0.2631486,
 0.24826024,
 0.24761842]

In [14]:
n_topics = 7
X = tfidf_documents

nmf = NMF(n_components=n_topics, random_state=0, init="nndsvd")
W = nmf.fit_transform(X) 
H = nmf.components_

top = 8
topic_index_max = n_topics

for topic_index in range(0, topic_index_max):
    top_indices = np.argsort(H[topic_index, :])[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append(feature_names[term_index])
    print("topic ", topic_index, top_terms)

topic  0 ['nan', 'passes', 'alive', 'probably', 'world', 'mum', 'inspire', 'dad']
topic  1 ['job', 'understand', 'friend', 'parents', 'wanted', 'home', 'stuff', 'mom']
topic  2 ['medication', 'meds', 'taking', 'adderall', 'doctor', 'vyvanse', 'dose', 'effects']
topic  3 ['sleep', 'night', 'tired', 'wake', 'bed', 'hours', 'asleep', 'sleeping']
topic  4 ['intrusive', 'compulsions', 'scared', 'head', 'real', 'fear', 'compulsion', 'happen']
topic  5 ['ptsd', 'trauma', 'therapy', 'therapist', 'happened', 'symptoms', 'panic', 'flashbacks']
topic  6 ['hate', 'fucking', 'shit', 'tired', 'fuck', 'wish', 'die', 'wanna']


In [15]:
from sklearn.feature_extraction.text import CountVectorizer

tf_vectorizer = CountVectorizer(stop_words='english')
tf_documents = tf_vectorizer.fit_transform(documents_train)
tf_feature_names = tf_vectorizer.get_feature_names_out()

print(tf_documents.shape)

(151288, 101960)


In [16]:
n_topics = 7

from sklearn.decomposition import LatentDirichletAllocation

# This could take a few minutes to run...

lda = LatentDirichletAllocation(n_components = n_topics, random_state=0)
lda.fit(tf_documents)
topic_models = lda.components_

In [17]:
num_top_words = 8

def display_topics(model, feature_names, no_top_words):
    for topic_idx, topic in enumerate(model.components_):
        term_list = [feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]
        print("topic %d:" % (topic_idx), term_list)


display_topics(lda, tf_feature_names, num_top_words)

topic 0: ['removed', 'im', 'job', 'school', 'work', 'just', 'year', 'college']
topic 1: ['im', 'adhd', 'ive', 'medication', 'diagnosed', 'just', 'meds', 'taking']
topic 2: ['im', 'like', 'ocd', 'just', 'feel', 'thoughts', 'dont', 'know']
topic 3: ['people', 'like', 'just', 'im', 'dont', 'things', 'time', 'think']
topic 4: ['im', 'just', 'like', 'feel', 'dont', 'ive', 'time', 'really']
topic 5: ['deleted', 'im', 'just', 'didnt', 'like', 'told', 'said', 'know']
topic 6: ['just', 'im', 'dont', 'life', 'want', 'like', 'feel', 'know']
